# Physics Informed Neural Networks

This notebook implements PINNs from Raissi et al. 2017. Specifically, the notebook adapts the code implementation of Data-Driven Solutions of Nonlinear Partial Differential Equations from https://github.com/maziarraissi/PINNs, and the code by Michael Ito, to solve the force-field equation for solar modulation of cosmic rays. Michael Ito's code adaption use the TF2 API where the main mechanisms of the PINN arise in the train_step function efficiently computing higher order derivatives of custom loss functions through the use of the GradientTape data structure. 

In this application, our PINN is $h(r, p) = \frac{\partial f}{\partial r} + \frac{RV}{3k} \frac{\partial f}{\partial p}$ where $k=\beta(p)k_1(r)k_2(r)$ and $\beta = \frac{p}{\sqrt{p^2 + M^2}}$. We will approximate $f(r, p)$ using a neural network.

We have no initial data, but our boundary data will be given by $f(r_{HP}, p) = \frac{J(r_{HP}, T)}{p^2} = \frac{(T+M)^\gamma}{p^2}$, where $r_{HP} = 120$ AU (i.e. the radius of Heliopause), $M=0.938$ GeV, $\gamma$ is between $-2$ and $-3$, and $T = \sqrt{p^2 + M^2} - M$. Or, vice versa, $p = \sqrt{T^2 + 2TM}$.


In [1]:
# Imports
import numpy as np
import tensorflow as tf
import sherpa
import pickle as pkl

tf.config.list_physical_devices(device_type=None)

2022-09-30 16:30:32.144165: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/apps/software/system/CUDA/11.0.2/nvvm/lib64:/opt/apps/software/system/CUDA/11.0.2/extras/CUPTI/lib64:/opt/apps/software/system/CUDA/11.0.2/lib:/opt/apps/software/lib/slurm-drmaa/1.1.3/lib:/opt/apps/software/lib/libevent/2.1.8/lib:/opt/apps/software/devel/PCRE/8.41-GCCcore-7.3.0/lib:/opt/apps/software/lang/Perl/5.28.0-GCCcore-7.3.0/lib:/opt/apps/software/tools/expat/2.2.5-GCCcore-7.3.0/lib:/opt/apps/software/lang/Python/2.7.15-foss-2018b/lib/python2.7/site-packages/numpy-1.14.5-py2.7-linux-x86_64.egg/numpy/core/lib:/opt/apps/software/lang/Python/2.7.15-foss-2018b/lib:/opt/apps/software/lib/libffi/3.2.1-GCCcore-7.3.0/lib64:/opt/apps/software/lib/libffi/3.2.1-GCCcore-7.3.0/lib:/opt/apps/software/math/GMP/6.1.2-GCCcore-7.3.0/lib:/opt/apps/so

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Load the Data

In [2]:
# Constants  
m = 0.938 # GeV/c^2
gamma = -3 # Between -2 and -3
size = 512 # size of r, T, p, and f_boundary

# Create intial data
T = np.logspace(np.log10(0.001), np.log10(1000), size).flatten()[:,None] # GeV
p = (np.sqrt((T+m)**2-m**2)).flatten()[:,None] # GeV/c
r = np.logspace(np.log10(119*150e6), np.log10(120*150e6), size).flatten()[:,None] # km
f_boundary = ((T + m)**gamma)/(p**2) # particles/(m^3 (GeV/c)^3)

# Take the log of all input data
r = np.log(r)
T = np.log(T)
p = np.log(p)
f_boundary = np.log(f_boundary)

# Domain bounds
lb = np.array([p[0], r[0]]) # (p, r) in (GeV, AU)
ub = np.array([p[-1], r[-1]]) # (p, r) in (GeV, AU)

# Flatten and transpose data for ML
P, R = np.meshgrid(p, r)
P_star = np.hstack((P.flatten()[:,None], R.flatten()[:,None]))

# Check inputs
# print(f'r: {r.shape}, p: {p.shape}, T: {T.shape}, f_boundary: {f_boundary.shape}, P_star: {P_star.shape}, lb: {lb}, ub:{ub}')

## PINN Class

The PINN class subclasses the Keras Model so that we can implement our custom fit and train_step functions.

In [20]:
'''
Description: Defines the class for a PINN model implementing train_step, fit, and predict functions. Note, it is necessary 
to design each PINN seperately for each system of PDEs since the train_step is customized for a specific system. 
This PINN in particular solves the force-field equation for solar modulation of cosmic rays. Once trained, the PINN can predict the solution space given 
domain bounds and the input space. 
'''
class PINN(tf.keras.Model):
    def __init__(self, inputs, outputs, lower_bound, upper_bound, p, r, f_boundary, size, n_samples=20000, n_boundary=50):
        super(PINN, self).__init__(inputs=inputs, outputs=outputs)
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.p = p
        self.r = r
        self.f_boundary = f_boundary
        self.n_samples = n_samples
        self.n_boundary = n_boundary
        self.size = size
        
    '''
    Description: A system of PDEs are determined by 2 types of equations: the main partial differential equations 
    and the boundary value equations. These two equations will serve as loss functions which 
    we train the PINN to satisfy. If a PINN can satisfy BOTH equations, the system is solved. Since there are 2 types of 
    equations (PDE, Boundary Value), we will need 2 types of inputs. Each input is composed of a spatial 
    variable 'r' and a momentum variable 'p'. The different types of (p, r) pairs are described below.
    
    Inputs: 
        p, r: (batchsize, 1) shaped arrays : These inputs are used to derive the main partial differential equation loss.
        Train step first feeds (p, r) through the PINN for the forward propagation. This expression is PINN(p, r) = f. 
        Next, the partials f_p and f_r are obtained. We utilize TF2s GradientTape data structure to obtain all partials. 
        Once we obtain these partials, we can compute the main PDE loss and optimize weights w.r.t. to the loss. 
        
        p_boundary, r_boundary : (boundary_batchsize, 1) shaped arrays : These inputs are used to derive the boundary value
        equations. The boundary value loss relies on target data (**not an equation**), so we can just measure the MAE of 
        PINN(p_boundary, r_boundary) = f_pred_boundary and boundary_f.
        
        f_boundary: (boundary_batchsize, 1) shaped arrays : This is the target data for the boundary value inputs
        
        alpha = weight on pinn_loss
        
    Outputs: sum_loss, pinn_loss, boundary_loss
    '''
    def train_step(self, p, r, p_boundary, r_boundary, f_boundary, alpha):
        with tf.GradientTape(persistent=True) as t2: 
            with tf.GradientTape(persistent=True) as t1: 
                # Forward pass P (PINN data)
                P = tf.concat((p, r), axis=1)
                f = self.tf_call(P)

                # Forward pass P_boundary (boundary condition data)
                P_boundary = tf.concat((p_boundary, r_boundary), axis=1)
                f_pred_boundary = self.tf_call(P_boundary)

                # Calculate boundary loss
                boundary_loss = tf.math.reduce_mean(tf.math.abs(f_pred_boundary - f_boundary))

            # Calculate first-order PINN gradients
            f_p = t1.gradient(f, p)
            f_r = t1.gradient(f, r)
            print(f'f_r: {f_r[-1]}, r: {tf.math.exp(r)[-1]/150e6}, f: {tf.math.exp(f)[-1]}')
            
            pinn_loss = self.pinn_loss(p, r, f_p, f_r)
            total_loss = alpha*pinn_loss + (1-alpha)*boundary_loss
        
        # Backpropagation
        gradients = t2.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        # Return losses
        return pinn_loss.numpy(), boundary_loss.numpy()
    
    '''
    Description: The fit function used to iterate through epoch * steps_per_epoch steps of train_step. 
    
    Inputs: 
        P_predict: (N, 2) array: Input data for entire spatial and temporal domain. Used for vizualization for
        predictions at the end of each epoch. Michael created a very pretty video file with it. 
        
        alpha: weight on pinn_loss
        
        batchsize: batchsize for (p, r) in train step
        
        boundary_batchsize: batchsize for (x_lower, t_boundary) and (x_upper, t_boundary) in train step
        
        epochs: epochs
        
        lr: learning rate
        
        size: size of the prediction data (i.e. len(p) and len(r))
        
        save: Whether or not to save the model to a checkpoint every 10 epochs
        
        load_epoch: If -1, a saved model will not be loaded. Otherwise, the model will be 
        loaded from the provided epoch
        
        lr_decay: If -1, learning rate will not be decayed. Otherwise, lr = lr_decay*lr if loss doesn't
        decrease for 3 epochs
        
        weight_change: If -1, alpha will not be changed. Otherwise, alpha = weight_change*alpha if loss 
        doesn't decrease for 3 epochs
        
        patience: Number of epochs to check whether loss has decreased before updating lr or alpha
        
        filename: Name for the checkpoint file
    
    Outputs: Losses for each equation (Total, PDE, Boundary Value), and predictions for each epoch.
    '''
    def fit(self, P_predict, alpha=1, batchsize=64, boundary_batchsize=16, epochs=20, lr=3e-3, size=256, 
            save=False, load_epoch=-1, lr_decay=-1, weight_change=-1, patience=3, filename=''):
        # If load == True, load the weights
        if load_epoch != -1:
            name = './ckpts/pinn_' + filename + '_epoch_' + str(load_epoch)
            self.load_weights(name)
        
        # Initialize losses as zeros
        steps_per_epoch = np.ceil(self.n_samples / batchsize).astype(int)
        total_pinn_loss = np.zeros((epochs, ))
        total_boundary_loss = np.zeros((epochs, ))
        predictions = np.zeros((size**2, 1, epochs))
        
        # For each epoch, sample new values in the PINN and boundary areas and pass them to train_step
        for epoch in range(epochs):
            # Compile
            opt = tf.keras.optimizers.Adam(learning_rate=lr)
            self.compile(optimizer=opt)

            # Reset loss variables
            sum_loss = np.zeros((steps_per_epoch,))
            pinn_loss = np.zeros((steps_per_epoch,))
            boundary_loss = np.zeros((steps_per_epoch,))
            
            # For each step, get PINN and boundary variables and pass them to train_step
            for step in range(steps_per_epoch):
                # Get PINN p and r variables via uniform distribution sampling between lower and upper bounds
                p = tf.Variable(tf.random.uniform((batchsize, 1), minval=self.lower_bound[0], maxval=self.upper_bound[0]))
                r = tf.Variable(tf.random.uniform((batchsize, 1), minval=self.lower_bound[1], maxval=self.upper_bound[1]))
                
                # Randomly sample boundary_batchsize from p_boundary and f_boundary
                p_idx = np.expand_dims(np.random.choice(self.f_boundary.shape[0], boundary_batchsize, replace=False), axis=1)
                p_boundary = self.p[p_idx]
                f_boundary = self.f_boundary[p_idx]
                
                # Create r_boundary array = r_HP
                upper_bound = np.zeros((boundary_batchsize, 1))
                upper_bound[:] = self.upper_bound[1]
                r_boundary = tf.Variable(upper_bound, dtype=tf.float32)
                
                # Pass variables through the model via train_step and get losses
                losses = self.train_step(p, r, p_boundary, r_boundary, f_boundary, alpha)
                pinn_loss[step] = losses[0]
                boundary_loss[step] = losses[1]
            
            # Calculate and print total losses for the epoch
            total_pinn_loss[epoch] = np.sum(pinn_loss)
            total_boundary_loss[epoch] = np.sum(boundary_loss)
            print(f'Training loss for epoch {epoch}: pinn: {total_pinn_loss[epoch]:.4f}, boundary: {total_boundary_loss[epoch]:.4f}, total: {(total_boundary_loss[epoch]+total_pinn_loss[epoch]):.4f}')
            
            # Predict
            predictions[:, :, epoch] = self.predict(P_predict, size)
            
            # Determine if loss has decreased since the last patience epoch
            if (epoch > patience):
                hasDecreased = False
                if (total_pinn_loss[epoch] + total_boundary_loss[epoch]) < (total_pinn_loss[epoch-patience] + total_boundary_loss[epoch-patience]):
                    hasDecreased = True
                        
                # If loss hasn't decreased for the past 2 epochs, decrease lr by lr_decay
                if (lr_decay != -1) & (not hasDecreased):
                    lr = lr_decay*lr

                # If pinn loss hasn't decreased for the past 2 epochs, increase alpha by weight_change
                if (weight_change != -1) & (not hasDecreased):
                    alpha = np.tanh(weight_change*alpha)

            # If the epoch is a multiple of 10, save to a checkpoint
            if (epoch%10 == 0) & (save == True):
                name = './ckpts/pinn_' + filename + '_epoch_' + str(epoch)
                self.save_weights(name, overwrite=True, save_format=None, options=None)
        
        # Return epoch losses
        return total_pinn_loss, total_boundary_loss, predictions
    
    # Predict for some P's the value of the neural network f(r, p)
    def predict(self, P, size, batchsize=2048):
        steps_per_epoch = np.ceil(P.shape[0] / batchsize).astype(int)
        preds = np.zeros((size**2, 1))
        
        # For each step calculate start and end index values for prediction data
        for step in range(steps_per_epoch):
            start_idx = step * 64
            
            # If last step of the epoch, end_idx is shape-1. Else, end_idx is start_idx + 64 
            if step == steps_per_epoch - 1:
                end_idx = P.shape[0] - 1
            else:
                end_idx = start_idx + 64
                
            # Pass prediction data through the model
            preds[start_idx:end_idx, :] = self.tf_call(P[start_idx:end_idx, :]).numpy()
        
        # Return f
        return preds
    
    def evaluate(self, ): 
        pass
    
    # pinn_loss calculates the PINN loss by calculating the MAE of the pinn function
    @tf.function
    def pinn_loss(self, p, r, f_p, f_r): # To-do: add loss pass-through!
        # Note: p and r are taken out of logspace for the PINN calculation
        p = tf.math.exp(p) # GeV/c
        r = tf.math.exp(r) # km
        V = 400 # 400 km/s
        m = 0.938 # GeV/c^2
        k_0 = 1e11 # km^2/s
        k_1 = k_0 * tf.math.divide(r, 150e6) # km^2/s
        k_2 = p # unitless, k_2 = p/p0 and p0 = 1 GeV/c
        R = p # GV
        beta = tf.math.divide(p, tf.math.sqrt(tf.math.square(p) + tf.math.square(m))) 
        k = beta*k_1*k_2
        
        # Calculate physics loss
        l_f = tf.math.reduce_mean(tf.math.abs(f_r + (tf.math.divide(R*V, 3*k) * f_p)))
        
        return l_f
    
    # tf_call passes inputs through the neural network
    @tf.function
    def tf_call(self, inputs): 
        return self.call(inputs, training=True)

In [22]:
# Neural network. Note: 2 inputs- (p, r), 1 output- f(r, p)
inputs = tf.keras.Input((2))
x_ = tf.keras.layers.Dense(79, activation='selu')(inputs)
x_ = tf.keras.layers.Dense(79, activation='selu')(x_)
x_ = tf.keras.layers.Dense(79, activation='selu')(x_)
x_ = tf.keras.layers.Dense(79, activation='selu')(x_)
outputs = tf.keras.layers.Dense(1, activation='linear')(x_) 

# Hyperparameters
alpha = 0.2902434203767448
lr = 3e-2
lr_decay = 0.9162384181161904
batchsize = 512
boundary_batchsize = 128
epochs = 100
save = False
load_epoch = -1
weight_change = 1.4426168198220866
patience = 3
filename = 'sherpa_id_11'

# Initialize and fit the PINN
pinn = PINN(inputs=inputs, outputs=outputs, lower_bound=lb, upper_bound=ub, p=p[:, 0], r=r[:, 0], 
            f_boundary=f_boundary[:, 0], size=size)
pinn_loss, boundary_loss, predictions = pinn.fit(P_predict=P_star, alpha=alpha, batchsize=batchsize, boundary_batchsize=boundary_batchsize,
                                                         epochs=epochs, lr=lr, size=size, save=save, load_epoch=load_epoch, lr_decay=lr_decay,
                                                         weight_change=weight_change, patience=patience, filename=filename)

f_r: [0.11243305], r: [119.888405], f: [0.24094352]
f_r: [-0.40531927], r: [119.46361], f: [2.4677431e-17]
f_r: [3.817642], r: [119.0201], f: [inf]
f_r: [0.8145855], r: [119.91813], f: [7.903894e+10]
f_r: [-0.18340082], r: [119.795135], f: [1.7358079e-07]
f_r: [-0.0682313], r: [119.48889], f: [2.481569e-07]
f_r: [0.02960273], r: [119.7675], f: [0.00191598]
f_r: [0.15711254], r: [119.47318], f: [0.08355512]
f_r: [0.17032117], r: [119.95199], f: [0.07672039]
f_r: [0.15539767], r: [119.88017], f: [2.2920258]
f_r: [0.81583107], r: [119.523315], f: [0.00038325]
f_r: [0.33480275], r: [119.85365], f: [3.5493188e-12]
f_r: [0.20989785], r: [119.78897], f: [20.128105]
f_r: [0.03791428], r: [119.26872], f: [8.505934e-08]
f_r: [0.20013623], r: [119.46314], f: [1.9118847e-13]
f_r: [1.1151339], r: [119.86371], f: [0.0181007]
f_r: [0.05704463], r: [119.995926], f: [16.250704]
f_r: [0.12915403], r: [119.94559], f: [4.4815038e-10]
f_r: [0.55110043], r: [119.98219], f: [0.00516541]
f_r: [0.15273924], r:

## Save the outputs to a pickle

In [23]:
# Save PINN outputs
with open('./figures/pickles/pinn_loss_' + filename + '.pkl', 'wb') as file:
    pkl.dump(pinn_loss, file)
    
with open('./figures/pickles/boundary_loss_' + filename + '.pkl', 'wb') as file:
    pkl.dump(boundary_loss, file)
    
with open('./figures/pickles/predictions_' + filename + '.pkl', 'wb') as file:
    pkl.dump(predictions, file)
    
# with open('./figures/pickles/f_boundary.pkl', 'wb') as file:
#     pkl.dump(f_boundary, file)
    
# with open('./figures/pickles/p.pkl', 'wb') as file:
#     pkl.dump(p, file)
    
# with open('./figures/pickles/T.pkl', 'wb') as file:
#     pkl.dump(T, file)
    
# with open('./figures/pickles/r.pkl', 'wb') as file:
#     pkl.dump(r, file)